In [11]:
import numpy as np # to handle matrix and data operation
import pandas as pd # to read csv and handle dataframe

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable

from sklearn.model_selection import train_test_split

In [12]:
df = pd.read_csv('/Users/taraapple/Desktop/train.csv')
print(df.shape)

(42000, 785)


In [13]:
y = df['label'].values
X = df.drop(['label'],1).values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [14]:
print(y_test.shape)

(6300,)


In [15]:
BATCH_SIZE = 32

torch_X_train = torch.from_numpy(X_train).type(torch.LongTensor)
torch_y_train = torch.from_numpy(y_train).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set.
torch_X_test = torch.from_numpy(X_test).type(torch.LongTensor)
torch_y_test = torch.from_numpy(y_test).type(torch.LongTensor) # data type is long

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

In [16]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(784,250)
        self.linear2 = nn.Linear(250,100)
        self.linear3 = nn.Linear(100,10)
    
    def forward(self,X):
        X = F.relu(self.linear1(X))
        X = F.relu(self.linear2(X))
        X = self.linear3(X)
        return F.log_softmax(X, dim=1)
 
mlp = MLP()
print(mlp)

MLP(
  (linear1): Linear(in_features=784, out_features=250, bias=True)
  (linear2): Linear(in_features=250, out_features=100, bias=True)
  (linear3): Linear(in_features=100, out_features=10, bias=True)
)


In [17]:
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters())#,lr=0.001, betas=(0.9,0.999))
    error = nn.CrossEntropyLoss()
    EPOCHS = 5
    model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

            # Total correct predictions
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
            #print(correct)
            if batch_idx % 50 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.item(), float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))
                

In [18]:
fit(mlp, train_loader)

Epoch : 0 [0/35700 (0%)]	Loss: 10.092637	 Accuracy:9.375%
Epoch : 0 [1600/35700 (4%)]	Loss: 0.393456	 Accuracy:69.669%
Epoch : 0 [3200/35700 (9%)]	Loss: 0.408826	 Accuracy:77.692%
Epoch : 0 [4800/35700 (13%)]	Loss: 0.527875	 Accuracy:81.002%
Epoch : 0 [6400/35700 (18%)]	Loss: 0.545816	 Accuracy:82.991%
Epoch : 0 [8000/35700 (22%)]	Loss: 0.554889	 Accuracy:84.562%
Epoch : 0 [9600/35700 (27%)]	Loss: 0.109636	 Accuracy:85.569%
Epoch : 0 [11200/35700 (31%)]	Loss: 0.083980	 Accuracy:86.485%
Epoch : 0 [12800/35700 (36%)]	Loss: 0.082152	 Accuracy:87.071%
Epoch : 0 [14400/35700 (40%)]	Loss: 0.204077	 Accuracy:87.618%
Epoch : 0 [16000/35700 (45%)]	Loss: 0.206722	 Accuracy:87.918%
Epoch : 0 [17600/35700 (49%)]	Loss: 0.756245	 Accuracy:88.198%
Epoch : 0 [19200/35700 (54%)]	Loss: 0.221217	 Accuracy:88.722%
Epoch : 0 [20800/35700 (58%)]	Loss: 0.201075	 Accuracy:88.988%
Epoch : 0 [22400/35700 (63%)]	Loss: 0.284588	 Accuracy:89.274%
Epoch : 0 [24000/35700 (67%)]	Loss: 0.130226	 Accuracy:89.660%
Epoch

In [19]:
def evaluate(model):
#model = mlp
    correct = 0 
    for test_imgs, test_labels in test_loader:
        #print(test_imgs.shape)
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f}% ".format( float(correct) / (len(test_loader)*BATCH_SIZE)))
evaluate(mlp)

Test accuracy:0.955% 


In [20]:
torch_X_train = torch_X_train.view(-1, 1,28,28).float()
torch_X_test = torch_X_test.view(-1,1,28,28).float()
print(torch_X_train.shape)
print(torch_X_test.shape)

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

torch.Size([35700, 1, 28, 28])
torch.Size([6300, 1, 28, 28])


In [21]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32,64, kernel_size=5)
        self.fc1 = nn.Linear(3*3*64, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv3(x),2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.view(-1,3*3*64 )
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
 
cnn = CNN()
print(cnn)

it = iter(train_loader)
X_batch, y_batch = next(it)
print(cnn.forward(X_batch).shape)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)
torch.Size([32, 10])


In [22]:
fit(cnn,train_loader)

Epoch : 0 [0/35700 (0%)]	Loss: 21.994940	 Accuracy:9.375%
Epoch : 0 [1600/35700 (4%)]	Loss: 2.122579	 Accuracy:15.686%
Epoch : 0 [3200/35700 (9%)]	Loss: 0.940327	 Accuracy:29.115%
Epoch : 0 [4800/35700 (13%)]	Loss: 1.053328	 Accuracy:42.136%
Epoch : 0 [6400/35700 (18%)]	Loss: 0.833530	 Accuracy:51.011%
Epoch : 0 [8000/35700 (22%)]	Loss: 0.741710	 Accuracy:57.134%
Epoch : 0 [9600/35700 (27%)]	Loss: 0.429058	 Accuracy:61.628%
Epoch : 0 [11200/35700 (31%)]	Loss: 0.206912	 Accuracy:64.984%
Epoch : 0 [12800/35700 (36%)]	Loss: 0.396779	 Accuracy:67.573%
Epoch : 0 [14400/35700 (40%)]	Loss: 0.216545	 Accuracy:69.886%
Epoch : 0 [16000/35700 (45%)]	Loss: 0.365804	 Accuracy:71.644%
Epoch : 0 [17600/35700 (49%)]	Loss: 0.168453	 Accuracy:73.344%
Epoch : 0 [19200/35700 (54%)]	Loss: 0.254368	 Accuracy:74.813%
Epoch : 0 [20800/35700 (58%)]	Loss: 0.402496	 Accuracy:75.845%
Epoch : 0 [22400/35700 (63%)]	Loss: 0.420994	 Accuracy:76.877%
Epoch : 0 [24000/35700 (67%)]	Loss: 0.547721	 Accuracy:77.830%
Epoch

In [23]:
evaluate(cnn)

Test accuracy:0.954% 
